In [1]:
import pandas as pd

In [2]:
import time

In [3]:
import get_kepler

In [4]:
import numpy as np
from astropy.timeseries import LombScargle
from astropy import units as u
import matplotlib.pyplot as plt

In [5]:
data = get_kepler.read_csv(5697805)

In [6]:
data["period"]

0.01

In [7]:
data["periods"]

[0.010210159726767505,
 0.010223750233830188,
 0.020434891953662427,
 0.010209625752077445,
 0.010208229051540754,
 0.010223602392182753,
 0.010223781532634615,
 0.010209699379121714,
 0.010224067233680165,
 0.010219627603662226,
 0.02043493632791364,
 0.010211784645562026,
 0.010222693735549222,
 0.010217250530153656,
 0.010221289537361384]